<h1>Data</h1>

In [1]:
%matplotlib inline
from mysql import connector
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

mysql_connector = connector.connect(host="127.0.0.1", database="aq", 
                                    user="root", password="!QAZ2wsx#EDC")
data = pd.read_sql("""SELECT close, volume FROM future_trade
                      WHERE code='%s' AND type='%s' """ % ("P", "5"), con=mysql_connector)
mysql_connector.close()
data["chg"] = data.close.diff()
data.iloc[0, 2] = 0

lag_rtn = 10
lag_vol = 10
y = data.chg[max(lag_rtn, lag_vol):]
y.index = range(len(y))
y = y.apply(lambda x: x > 0)
X = pd.DataFrame(columns=range(lag_rtn + lag_vol))
for idx, row in data.iterrows():
    if idx >= lag_rtn and idx >= lag_vol:
        vol = data.iloc[(idx - lag_vol):idx, 1]
        rtn = data.iloc[(idx - lag_rtn):idx, 2]
        xrow = pd.concat([rtn, vol], ignore_index=True)
        X = X.append(xrow, ignore_index=True)

print("Std:", data.chg.std())
print("Mean:", data.chg.mean())

Std: 11.611485198066328
Mean: 0.000990136715031037


# Logistic Regression

In [3]:
from sklearn.linear_model import LogisticRegression
k_fold = 50
stride = round(len(data.chg)/k_fold)
for i in range(0, k_fold):
    start = i*stride
    stop = i*stride+stride-1
    if (i == k_fold-1):
        stop = len(data.chg)
    X_test = X[start:stop]
    y_test = y[start:stop]
    X_train = pd.concat([X[0:start-1], X[stop:]])
    y_train = pd.concat([y[0:start-1], y[stop:]])    
    model = LogisticRegression()
    model.fit(X_train, y_train)
    print("i=%s, %.2f%s" % (i, np.sum(model.predict(X_test) == y_test)*100/stride, "%"))    

i=0, 62.48%
i=1, 61.24%
i=2, 64.48%
i=3, 65.05%
i=4, 66.95%
i=5, 62.95%
i=6, 68.10%
i=7, 63.43%
i=8, 63.71%
i=9, 64.29%
i=10, 60.67%
i=11, 61.90%
i=12, 65.24%
i=13, 65.33%
i=14, 68.19%
i=15, 63.62%
i=16, 62.10%
i=17, 60.95%
i=18, 65.43%
i=19, 64.10%
i=20, 61.33%
i=21, 65.90%
i=22, 62.48%
i=23, 61.52%
i=24, 58.76%
i=25, 59.81%
i=26, 60.48%
i=27, 58.57%
i=28, 58.19%
i=29, 60.38%
i=30, 60.57%
i=31, 59.71%
i=32, 64.76%
i=33, 55.43%
i=34, 54.86%
i=35, 60.48%
i=36, 59.62%
i=37, 58.48%
i=38, 55.62%
i=39, 57.81%
i=40, 57.52%
i=41, 57.52%
i=42, 56.19%
i=43, 58.10%
i=44, 55.43%
i=45, 54.00%
i=46, 55.71%
i=47, 57.33%
i=48, 58.57%
i=49, 54.76%


# Navie Bayes

In [ ]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(X_train, y_train)
expected = y_test
"%.2f%s" % (np.sum(model.predict(X_test) == y_test)*100/length, "%")